In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
from faker import Faker
import random

from snowflake.snowpark.context import get_active_session


session = get_active_session()

In [ ]:
#Synthetically create a dataframe of customers, products, contract val, and received revenue
fake = Faker()

def generate_customer_data(num_customers=100):
    """Generates mock customer data."""

    data = []
    product_list =["Cortex Search","Cortex Analyst",
                "DocAI","Feature Store",
                "Model Registry","Cortex Agents"]
    
    for _ in range(num_customers):
        customer_name = fake.company()
        products = random.sample(product_list
,
            random.randint(1, 4),
        )
        total_contract_value = round(random.uniform(1000, 100000), 2)
        revenue_received = round(random.uniform(0, total_contract_value), 2)

        data.append(
            {
                "Customer": customer_name,
                "Total_Contract_Value": total_contract_value,
                "Revenue_Received": revenue_received,
                "Products_List": products,

            }
        )
    df = pd.DataFrame(data)

    for product in product_list:
        df[f"{product.replace(' ', '_')}_User"] = df['Products_List'].apply(lambda x: 'Y' if product in x else 'N')

    # df = df[["Customer","Products_List", "Cortex_Search_user", "Cortex_Analyst_user", 
    # "DocAI_user", "Feature_Store_user", "Model_Registry_user", "Cortex_Agents_user",
    # "Total_Contract_Value", "Revenue_Received"]]

    # df = df[["Customer","Products_List","Model_Registry_User",
    #         "Cortex_Search_User","Feature_Store_User","Cortex_Agents_User",
    #         "Total_Contract_Value","Revenue_Received"]]

    return df

# Generate and display the DataFrame
df = generate_customer_data()
df.head()
df.columns

In [ ]:
df.head()

In [ ]:
session.write_pandas(df, "CUSTOMER_PRODUCT_DATA", auto_create_table=True, quote_identifiers = False, overwrite=True)

In [ ]:
SHOW TABLES;

In [ ]:
CREATE OR REPLACE STAGE SEMANTIC DIRECTORY = ( ENABLE = true );

# We will now create the semantic model to query our new data with natural language

* To do so, first go to the AI Studio in snowsight and click the Cortex Analyst Tab

* Choose the appropriate database and schema the select the SEMANTIC stage we just created

* Click Create New

* Fill out the Description - 
    * "Semantic model containing information about customer product data including the customer name, which products they currently use, their total contract value, and the amount of revenue we have received from the customer"

* Select the CUSTOMER_PRODUCT_DATA table and select all columns


In [ ]:
data = session.table("CUSTOMER_PRODUCT_DATA")
data.show(3)

In [ ]:
## Start testing cortex analyst api call here
from typing import List, Dict, Optional
import _snowflake


class CortexAnalyst():
    def __init__(self, db: str, schema: str, stage: str, semantic_model_file_path: str):
        self.db = db
        self.schema = schema
        self.stage = stage
        self.semantic_model_file_path = semantic_model_file_path


    # @instrument
    def send_message(self, prompt: str) -> dict:

        """Calls the REST API and returns the response."""
        
        messages = []
        messages.append({"role": "user", "content": [{"type": "text", "text": prompt}]})
        request_body = {
            "messages": messages, #need to wrap in a list?
            "semantic_model_file": f"@{self.db}.{self.schema}.{self.stage}/{self.semantic_model_file_path}",
        }

        print(request_body)

        resp = _snowflake.send_snow_api_request(
            "POST",
            f"/api/v2/cortex/analyst/message",
            {},
            {},
            request_body,
            {},
            30000,
        )
        if resp["status"] < 400:
            return json.loads(resp["content"])
        else:
            # messages.pop()
            raise Exception(
                f"Failed request with status {resp['status']}: {resp}"
            )
    # @instrument
    def process_message(self, prompt: str) -> None:
        """Processes a message and adds the response to the chat."""
        messages=[]
        messages.append(
            {"role": "user", "content": [{"type": "text", "text": prompt}]}
        )
        with st.chat_message("user"):
            st.markdown(prompt)
        with st.chat_message("assistant"):
            with st.spinner("Generating response..."):
                # response = "who had the most rec yards week 10"
                response = self.send_message(prompt=prompt)
                request_id = response["request_id"]
                content = response["message"]["content"]
                messages.append(
                    {**response['message'], "request_id": request_id}
                )
                self.display_content(content=content, request_id=request_id)  # type: ignore[arg-type]
        return response
    
    # @instrument
    def display_content(self,
        content: List[Dict[str, str]],
        request_id: Optional[str] = None,
        message_index: Optional[int] = None,
    ) -> None:
        """Displays a content item for a message."""
        PROMPT = prompt
        message_index = message_index or len(PROMPT)
        if request_id:
            with st.expander("Request ID", expanded=False):
                st.markdown(request_id)
        for item in content:
            if item["type"] == "text":
                st.markdown(item["text"])
            elif item["type"] == "suggestions":
                with st.expander("Suggestions", expanded=True):
                    for suggestion_index, suggestion in enumerate(item["suggestions"]):
                        if st.button(suggestion, key=f"{message_index}_{suggestion_index}"):
                            st.session_state.active_suggestion = suggestion
            elif item["type"] == "sql":
                self.display_sql(item["statement"])

    # @instrument
    def display_sql(self, sql: str) -> None:
        with st.expander("SQL Query", expanded=False):
            st.code(sql, language="sql")
        with st.expander("Results", expanded=True):
            with st.spinner("Running SQL..."):
                session = get_active_session()
                df = session.sql(sql).to_pandas()
                if len(df.index) > 1:
                    data_tab, line_tab, bar_tab = st.tabs(
                        ["Data", "Line Chart", "Bar Chart"]
                    )
                    data_tab.dataframe(df)
                    if len(df.columns) > 1:
                        df = df.set_index(df.columns[0])
                    with line_tab:
                        st.line_chart(df)
                    with bar_tab:
                        st.bar_chart(df)
                else:
                    st.dataframe(df)

        return df.to_markdown(index=False)

CA = CortexAnalyst(db='TR_MULTI_AGENT', schema='PUBLIC', stage='SEMANTIC', semantic_model_file_path='customer_product_data_model.yaml')

In [ ]:
CA.process_message(prompt='Show me all the products for Allen and Sons')

In [ ]:
import json

prompt = 'Show me all the products for Allen and Sons'

messages = []

messages.append(
    {"role": "user", "content": [{"type": "text", "text": prompt}]}
)

request_body = {
    "messages": messages,
    "semantic_model_file": f"@TR_MULTI_AGENT.PUBLIC.SEMANTIC/customer_product_data_model.yaml",
}

print (request_body)
resp = _snowflake.send_snow_api_request(
    "POST",
    f"/api/v2/cortex/analyst/message",
    {},
    {},
    request_body,
    {},
    30000,
)

# resp
if resp["status"] < 400:
    print( json.loads(resp["content"]))
else:
    raise Exception(
        f"Failed request with status {resp['status']}: {resp}"
    )

In [ ]:
session.sql("WITH __customer_product_data AS (\n  SELECT\n    customer,\n    products_list\n  FROM tr_multi_agent.public.customer_product_data\n)\nSELECT DISTINCT\n  CAST(f.value AS TEXT) AS product_name\nFROM __customer_product_data, TABLE(FLATTEN(products_list)) AS f\nWHERE\n  customer = 'Allen and Sons'\nORDER BY\n  product_name\n -- Generated by Cortex Analyst\n;").collect()[0][0]

In [ ]:
from snowflake.snowpark import functions as F
import pandas as pd
import random
from datetime import datetime, timedelta

# Define parameters
num_orders = 10000  # Number of orders to generate

# Sample data
customers = ["Alpha Corp", "Beta Ltd", "Gamma Inc", "Delta LLC", "Epsilon SA"]
products = ["Steel Rods", "Copper Wires", "Aluminum Sheets", "Brass Fittings", "Iron Pipes"]
statuses = ["Pending", "Shipped", "Delivered", "Cancelled"]

# Generate random sales orders
orders = []
start_date = datetime(2024, 1, 1)

for i in range(num_orders):
    customer_id = f"ORD{1000 + i}"
    customer = random.choice(customers)
    product = random.choice(products)
    quantity = random.randint(10, 500)
    unit_price = round(random.uniform(5.0, 50.0), 2)
    total_price = round(quantity * unit_price, 2)
    order_date = start_date + timedelta(days=random.randint(0, 365))
    country = random.choice(countries)
    status = random.choice(statuses)
    
    orders.append([order_id, customer, product, quantity, unit_price, total_price, order_date, country, status])

# Create DataFrame
df_orders = pd.DataFrame(orders, columns=[
    "ORDER_ID", "CUSTOMER", "PRODUCT", "QUANTITY", "UNIT_PRICE", "TOTAL_PRICE", "ORDER_DATE", "COUNTRY", "STATUS"
])

customer_orders = session.write_pandas(df_orders[['CUSTOMER','ORDER_ID']].drop_duplicates(), table_name='CUSTOMER_ORDERS', overwrite=True)
orders = session.create_dataframe(df_orders[['ORDER_ID','ORDER_DATE','PRODUCT','QUANTITY','COUNTRY','STATUS']].drop_duplicates())
orders = orders.with_column("ORDER_DATE", col("ORDER_DATE").cast(T.DateType()))
orders.write.mode("overwrite").save_as_table('ORDERS', mode='overwrite')
products = df_orders[['PRODUCT','UNIT_PRICE']].groupby('PRODUCT').max().reset_index()
products = session.write_pandas(products, table_name='PRODUCTS', overwrite=True)

In [ ]:
from snowflake.cortex import complete
from trulens.core.otel.instrument import instrument
from trulens.otel.semconv.trace import SpanAttributes

class Multi_Agent_App:

    def agent_1_cortex_analyst(self, client: str) -> list:
        """
        takes client and inserts into predefined prompt and passes to cortex analyst to return list of products client uses today
        """
        request_body = {
            "messages": [f"Show me all products used by {client} today"],
            "semantic_model_file": f"@{DATABASE}.{SCHEMA}.{STAGE}/{FILE}",
        }
        resp = _snowflake.send_snow_api_request(...)

        products = session.sql(resp).collect()

        return products
    
    def agent_2_web_serach(self, client: str) -> list:
        """ 
        calls api to google client name and return top three new articles. Use bs4 or similar library to parse web results
        take parsed web result and pass to cortex LLM for summariziation
        """

        

    return cortex_llm_summary


    def agent_3_web_serach(self, client: str) -> list:
        """ 
        calls api to google client name and return top three articles. Use bs4 or similar library to parse web results
        take parsed web result and pass to cortex LLM for summariziation
        """
    return cortex_llm_summary

    def agent_4_prep_pitch(self, client: str) -> list:
        """ 
        calls api to google client name and return top three articles. Use bs4 or similar library to parse web results
        take parsed web result and pass to cortex LLM for summariziation
        """
        products = self.agent_1_cortex_analyst(client)
        news = self.agent_2_web_search(client)
        news = self.agent_2_web_search(client)
        prep_pitch = complete('claude-3-5-sonnet', f"prepare a product pitch based on the following products being used and the{products}")
    return cortex_llm_summary




rag = RAG()